# PROBABILE STRUTTURA
1. Caricamento dei file audio
2. Parsing del filename → estrazione emotion_id
3. Creazione DataFrame per vedere se tutto è corretto  ← QUI metti il codice
4. Generazione dei mel-spectrogram
5. Decisione dei parametri (n_mels, hop, win, durata, padding)
6. Test di un paio di augmentations


In [2]:
import os
os.getcwd()


'c:\\Users\\franc\\Politecnico Di Torino Studenti Dropbox\\Francesca Melloni\\POLITO\\2°anno, I semestre\\ML\\Project ML\\SER-Machine-Learning-Project\\notebooks'

In [3]:
import sys
sys.path.append("../src")

import pandas as pd
import os
from glob import glob
from preprocessing.dataset import extract_emotion_label, EMOTION_MAP

audio_files = glob("../data/Actor_*/*.wav", recursive=True)

rows = []

for f in audio_files:
    filename = os.path.basename(f)
    parts = filename.split('-')

    emotion_id = parts[2]
    emotion_label = EMOTION_MAP[emotion_id]

    actor = parts[-1].split('.')[0]  # ultimo numero prima del .wav

    rows.append([f, emotion_id, emotion_label, actor])

df = pd.DataFrame(rows, columns=["filepath", "emotion_id", "emotion_label", "actor"])
df

,filepath,emotion_id,emotion_label,actor
0,../data\Actor_01\03-01-01-01-01-01-01.wav,01,neutral,01
1,../data\Actor_01\03-01-01-01-01-02-01.wav,01,neutral,01
2,../data\Actor_01\03-01-01-01-02-01-01.wav,01,neutral,01
3,../data\Actor_01\03-01-01-01-02-02-01.wav,01,neutral,01
4,../data\Actor_01\03-01-02-01-01-01-01.wav,02,calm,01
...,...,...,...,...
1435,../data\Actor_24\03-01-08-01-02-02-24.wav,08,surprised,24
1436,../data\Actor_24\03-01-08-02-01-01-24.wav,08,surprised,24
1437,../data\Actor_24\03-01-08-02-01-02-24.wav,08,surprised,24
1438,../data\Actor_24\03-01-08-02-02-01-24.wav,08,surprised,24
